# Example of workflow with the EnergyScope model

In [1]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
ecoinvent_version = '3.9.1' # choose the ecoinvent version you wish to use
esm_location = 'CA-QC' # choose the version of energyscope for which you want to generate metrics
esm_year = 2020 # choose the year of the energyscope snapshot model or transition pathway time-step
regionalized_database = True # set to True if you want to use your regionalized version of ecoinvent

In [3]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}')  # put the name of your brightway project here

In [4]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_csv(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
impact_abbrev = pd.read_csv('../dev/lcia/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
premise_changes = pd.read_csv('../dev/data/premise_change_report.csv')

## Add CPC categories to the premise database

In [5]:
if regionalized_database:
    name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_remind_SSP2-Base_{esm_year}+truck_carculator regionalized"
else:
    name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_remind_SSP2-Base_{esm_year}"
name_premise_with_CPC_db = name_premise_db + '_with_CPC'

In [6]:
if name_premise_with_CPC_db not in bd.databases:
    premise_db = load_extract_db(name_premise_db, create_pickle=True)
    create_new_database_with_CPC_categories(db=premise_db, new_db_name=name_premise_with_CPC_db)

## Relinking the mapping file with a premise database

In [7]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db, create_pickle=True)

In [8]:
if not regionalized_database:
    pass
elif regionalized_database & (esm_location == 'CA-QC'):
    mapping['Database'] = len(mapping) * [name_premise_with_CPC_db]
else:
    mapping['Database'] += ' regionalized'

In [9]:
mapping = change_mapping_year(mapping, 2020, esm_year)

The mapping file is already for the year 2020


In [10]:
mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...


In [11]:
base_db = load_multiple_databases(list(mapping['Database'].unique()))
unlinked = test_mapping_file(mapping, base_db)

Mapping successfully linked to the database


In [12]:
mapping_linked_to_premise = create_complementary_database(mapping, premise_db_with_CPC, name_premise_db+f'_comp_{esm_location}', premise_changes)

The complementary database did not have to be created


In [13]:
if name_premise_db+f'_comp_{esm_location}' in bd.databases:
    premise_comp_db = load_extract_db(name_premise_db+f'_comp_{esm_location}') # complementary database without CPC categories
    create_new_database_with_CPC_categories(db=premise_comp_db, new_db_name=name_premise_db+f'_comp_{esm_location}')
    premise_comp_db = load_extract_db(name_premise_db+f'_comp_{esm_location}') # complementary database with CPC categories
    main_db = premise_db_with_CPC + premise_comp_db
else:
    main_db = premise_db_with_CPC

## Change location according to user ranking

In [14]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CAN', # Canada in IMAGE
        'CA-ON', # Other canadian provinces 
        'CA-AB',
        'CA-BC',
        'CA-MB',
        'CA-NB',
        'CA-NF',
        'CA-NS',
        'CA-NT',
        'CA-NU',
        'CA-PE',
        'CAZ', # Canada - Australia - New Zealand in REMIND
        'RNA', # North America
        'US', # United States
        'USA', # United States in REMIND and IMAGE
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH', 
        'RER', 
        'IAI Area, EU27 & EFTA',
        'NEU',
        'EUR',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

In [15]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']  
elif esm_location == 'CH':
    accepted_locations = ['CH'] 
else:
    accepted_locations = ['GLO', 'RoW'] 

In [16]:
# Update mapping dataframe
mapping_linked_to_premise = change_location_mapping_file(mapping_linked_to_premise, my_ranking, premise_db_with_CPC, esm_location)

In [17]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020+t...


In [18]:
# mapping_linked_to_premise.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv', index=False)

## Perform double-counting removal

In [19]:
mapping_linked_to_premise = mapping_linked_to_premise[mapping_linked_to_premise['Type'] != 'Flow']

In [20]:
new_db_name = f'energyscope_{esm_location}_{esm_year}'

In [21]:
regionalize_foregrounds = True # Set to True if you want to regionalize foregrounds

In [22]:
mapping_linked_to_premise_new_code = create_esm_database(
    mapping=mapping_linked_to_premise,
    model=model,
    tech_specifics=technology_specifics,
    technology_compositions=technology_compositions,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=main_db,
    esm_db_name=new_db_name,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    target_region=esm_location,
    locations_ranking=my_ranking,
    results_path_file=f'results/energyscope_{esm_location}/{esm_year}/',
)

AEC_OG
AFC
AFC_OG
ALKALINE_ELECTROLYSIS
AL_MAKING
AL_MAKING_HR
AN_DIG
AN_DIG_SI
ATR
ATR_CCS
BIOGAS_ATR
BIOGAS_ATR_CCS
BIOGAS_BIOMETHANE
BIOGAS_SMR
BIOGAS_SMR_CCS
BIOMASS_ETHANOL
BIOMASS_GAS_EF_H2
BIOMASS_GAS_EF_H2_CCS
BIOMASS_GAS_FB_H2
BIOMASS_GAS_FB_H2_CCS
BUS_BIODIESEL_B100_SD
BUS_BIODIESEL_B20_SD
BUS_CNG_SD
BUS_DIESEL_SD
BUS_EV_SD
BUS_FC_CH4_SD
BUS_FC_H2_SD
BUS_HY_DIESEL_SD
BUS_PROPANE_SD
CAR_BIODIESEL_B100_ELD
CAR_BIODIESEL_B100_LD
CAR_BIODIESEL_B100_MD
CAR_BIODIESEL_B100_SD
CAR_BIODIESEL_B20_ELD
CAR_BIODIESEL_B20_LD
CAR_BIODIESEL_B20_MD
CAR_BIODIESEL_B20_SD
CAR_CNG_ELD
CAR_CNG_LD
CAR_CNG_MD
CAR_CNG_SD
CAR_DIESEL_ELD
CAR_DIESEL_LD
CAR_DIESEL_MD
CAR_DIESEL_SD
CAR_ETOH_E10_ELD
CAR_ETOH_E10_LD
CAR_ETOH_E10_MD
CAR_ETOH_E10_SD
CAR_ETOH_E85_ELD
CAR_ETOH_E85_LD
CAR_ETOH_E85_MD
CAR_ETOH_E85_SD
CAR_EV_ELD
CAR_EV_LD
CAR_EV_MD
CAR_EV_SD
CAR_FC_CH4_ELD
CAR_FC_CH4_LD
CAR_FC_CH4_MD
CAR_FC_CH4_SD
CAR_FC_H2_ELD
CAR_FC_H2_LD
CAR_FC_H2_MD
CAR_FC_H2_SD
CAR_GASOLINE_ELD
CAR_GASOLINE_LD
CAR_GASOLINE_MD

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 07/15/2024 18:48:43
  Finished: 07/15/2024 18:48:45
  Total time elapsed: 00:00:02
  CPU %: 44.80
  Memory %: 14.82


## Computing the LCA metrics

In [23]:
esm_db = load_extract_db(new_db_name)

Getting activity data


100%|██████████| 1440/1440 [00:00<00:00, 144908.78it/s]


Adding exchange data to activities


100%|██████████| 36841/36841 [00:00<00:00, 38498.41it/s]


Filling out exchange data


100%|██████████| 1440/1440 [00:01<00:00, 730.81it/s] 


In [24]:
R_long = compute_impact_scores(
    esm_db=esm_db,
    mapping=mapping_linked_to_premise_new_code,
    technology_compositions=technology_compositions,
    # methods=['IMPACT World+ Midpoint 2.0.1', 'IMPACT World+ Damage 2.0.1', 'IMPACT World+ Footprint 2.0.1'],
    methods=['IMPACT World+ Midpoint 2.0.1_regionalized', 'IMPACT World+ Damage 2.0.1_regionalized'],
    # methods=['PB LCIA'],
    unit_conversion=unit_conversion,
    lifetime=lifetime,
)

In [25]:
R_long.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv', index=False)

## Convert the results in AMPL format

In [5]:
# R_long = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv')

In [6]:
refactor = 1e-2 # scaling factor to have the highest operation and construction metrics to the same order of magnitude (avoid having too many zeros)
# lcia_method = 'IMPACT World+ Damage 2.0.1 - Total only'
lcia_method = 'IMPACT World+ Damage 2.0.1_regionalized - Total only'
# lcia_method = 'PB LCIA'

### Create the .dat file

In [7]:
normalize_lca_metrics(
    R=R_long,
    f_norm=1e6,
    mip_gap=1e-6,
    refactor=refactor,
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
)

### Create the .mod file

In [8]:
gen_lcia_obj(
    lcia_method=lcia_method,
    refactor=refactor,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
)